<a href="https://colab.research.google.com/github/Nouf-Alabbasi/oKUmura_AI_Telecom_challenge/blob/Testing_N/No_router_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Test_3
run all cells with ⚫

##⚫ install req

In [1]:
!pip install streamlit
!pip install langchain
!pip install langchain langchain-community chromadb jq
!pip install openAI
!pip install langchain-openai
!pip install faiss-gpu
!pip install python-docx
!pip install unstructured
!pip install --quiet langchain_experimental langchain_openai
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.6/657.6 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

##⚫ setup

In [47]:
import streamlit as st
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
import os
from google.colab import userdata
import openai
from docx import Document
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

import json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.vectorstores.faiss import FAISS
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm

In [3]:
# Set your OpenAI API key
openai.api_key = userdata.get('OpenAI')
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI')



In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##chunking

##semantic chunking

In [ ]:
from llama_index.core import SimpleDirectoryReader

from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)
from llama_index.embeddings.openai import OpenAIEmbedding

def chunk_doc(file_path):
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    embed_model = OpenAIEmbedding()
    splitter = SemanticSplitterNodeParser(
        buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
    )

    # also baseline splitter
    base_splitter = SentenceSplitter(chunk_size=512)
    nodes = splitter.get_nodes_from_documents(documents)
    filename = os.path.basename(file_path)

    chunks = [{"text": node.get_content(), "source": filename} for node in nodes]
    print(chunks)
    return chunks

def process_all_documents(folder_path):
    all_chunks = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".docx"):
            file_path = os.path.join(folder_path, filename)
            chunks = chunk_doc(file_path)
            all_chunks.extend(chunks)
    return all_chunks

# Set the folder path to your "Documents" directory
folder_path = "/content/3GPP-Release18/Documents"
all_document_chunks = process_all_documents(folder_path)

# Example: Print the first chunk of each document
for chunk in all_document_chunks:
    print(chunk)

Document_ds = [chunk_doc('/content/3GPP-Release18/Documents/' + doc['id']) for doc in document_ds]

##⚫ recursive chunking

In [48]:

# Cache the function to load and process PDF documents
@st.cache(allow_output_mutation=True)

def load_and_process_docx(docx_folder_path):
  documents = []
  file_list = os.listdir(docx_folder_path)
  i = 0
  for file in tqdm(file_list):
    if file ends with ".docx":
      print(file)
      loader = UnstructuredWordDocumentLoader(docx_folder_path+file)
      data = loader.load()
      documents.append(data)

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
  for doc in documents:
    splits = text_splitter.split_documents(doc)
  # docx_loader = DirectoryLoader(folder_path, glob="./*.docx", loader_cls=UnstructuredWordDocumentLoader)
  # documents += docx_loader.load()
  # text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
  # texts = text_splitter.split_documents(documents)

  return splits

# Cache the function to initialize the vector store with documents
@st.cache(allow_output_mutation=True)
def initialize_vectorstore(_splits):
    return FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=userdata.get('OpenAI')))

2024-06-25 12:03:53.774 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.

2024-06-25 12:03:53.780 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.



In [49]:
folder_path = "/content/drive/MyDrive/KU/RAG_telco_files/documents/"

In [ ]:
splits = load_and_process_docx(folder_path)
# vectorstore = initialize_vectorstore(splits)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(splits, embeddings)

db.save_local("/content/drive/MyDrive/KU/RAG_telco_files/faiss_index")

# db = FAISS.load_local("/content/drive/MyDrive/KU/RAG_telco_files/faiss_index", embeddings)

# # to load the db
# db.load(faiss_file_path="/content/drive/MyDrive/KU/RAG_telco_files/", sql_url= "sqlite:///")

  0%|          | 0/564 [00:00<?, ?it/s]

29591-i40.docx


  0%|          | 1/564 [00:18<2:56:10, 18.77s/it]

24368-i20.docx


  0%|          | 2/564 [00:26<1:52:39, 12.03s/it]

...

28533-i00.docx


 87%|████████▋ | 493/564 [2:03:58<19:50, 16.76s/it]

37483-i00.docx


 88%|████████▊ | 494/564 [2:04:42<29:07, 24.97s/it]

36413-i00.docx


##⚫ retrieval

In [14]:
retriever = db.as_retriever()

In [15]:
docs = retriever.invoke("What does the AF request include when invoking the Nnef_DNAIMapping_Subscribe service?")

In [16]:

# for d in docs:
#   Retrievals.append(d.page_content)

# Retrievals
filename = docs[0].metadata['source'].split('/')[-1]
context = docs[0].page_content

##⚫ generator

##LLM(*Phi*-2)

In [19]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

In [20]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
def phi2(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)
    # outputs = model.generate(**inputs, max_length=2048)
    outputs = model.generate(**inputs, max_new_tokens=500, repetition_penalty=1.15)#, do_sample=True, top_p=0.90, num_return_sequences=3)
    text = tokenizer.batch_decode(outputs)[0]

    return text

prompt = '''def print_prime(n):
   """
   Print all primes between 1 and n
   """'''

phi2(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'def print_prime(n):\n   """\n   Print all primes between 1 and n\n   """\n\n   for i in range(2, n+1):\n       isPrime = True\n       for j in range(2, int(i**0.5)+1):\n           if (i % j) == 0:\n               isPrime = False\n               break\n       if isPrime:\n           print(i)\n```\n\n## Section 2: Prime Factorization\n\nThe prime factorization of a number involves expressing the number as a product of its prime factors. This process can be achieved using a while loop to continuously divide the given number by the smallest possible prime until it becomes one. \n\nHere\'s an example function that calculates the prime factorization of a number:\n\n```python\nimport math\n\ndef prime_factors(n):\n    """\n    Return list of prime factors for a given integer\n    """\n    factors = []\n    while n % 2 == 0:  # While n is divisible by 2\n        factors.append(2),\n        n /= 2\n    for i in range(3,int(math.sqrt(n))+1,2): # Check odd numbers from 3 up to sqrt(n)\n        w

##generate

In [28]:
query = "What does the AF request include when invoking the Nnef_DNAIMapping_Subscribe service? [3GPP Release 18]"
content = ""
docs.pop()
for index, elem in enumerate(docs):
  content = content + "\n" + elem.page_content
  print(elem.metadata['source'].split('/')[-1])

o = ["option 1 EAS address information and DNN", "option 2 EAS address information and S-NSSAI", "option 3 EAS address information, DNN, S-NSSAI, and AF Identifier", "option 4 EAS address information only"]
options = "\n".join(o)


syst_prompt = f"""
Instruct:
Please provide the answers to the following multiple choice question.
{query}

Considering the following context:
{content}

Please provide the answers to the following multiple choice question.
{query}
The output should be in the format: Option <Option id>

Options:
Write only the option number corresponding to the correct answer:\n{options}

Output:
"""

29591-i40.docx
29591-i40.docx


In [29]:
predicted_answers_str = phi2(syst_prompt)
print(f"predicted_answers_str {predicted_answers_str}")
predicted_answers_str = predicted_answers_str.split("Output:")[-1]
print(predicted_answers_str)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


predicted_answers_str 
Instruct:
Please provide the answers to the following multiple choice question.
What does the AF request include when invoking the Nnef_DNAIMapping_Subscribe service? [3GPP Release 18]

Considering the following context:

4.6.2	Service Operations

4.6.2.1	Introduction

Service operations defined for the Nnef_DNAIMapping Service are shown in table 4.6.2.1-1.

Table 4.6.2.1-1: Nnef_DNAIMapping Service Operations

Service Operation Name Description Initiated by Nnef_DNAIMapping_Subscribe This service operation is used by an NF service consumer to explicitly subscribe the notification of DNAI Mapping Information. NF service consumer Nnef_DNAIMapping_Unsubscribe This service operation is used by an NF service consumer to explicitly unsubscribe the notification of DNAI Mapping Information. NF service consumer Nnef_DNAIMapping_Notify This service operation is used by the NEF to provide Update DNAI Mapping Information to the NF service consumer. NEF

4.6.2.2	Nnef_DNAIMap

#drafts

##Test RAG with no router + using langchain from [link](https://github.com/rasyosef/rag-with-phi-2-and-langchain/blob/main/app.py)

###install req

In [ ]:
!pip install gradio
!pip install langchain
!pip install langchain-community
!pip install transformers
!pip install torch
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 619.0 kB/s 

In [ ]:
!pip install accelerate
# !pip -qqq install bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.9 MB/s eta 0:00:00


In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.0 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=a95d529ed6c3fc817148ecd45741ca98c008c08005aad65ba43ca50b1396892f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers torch langchain langchain-community


###setup

In [ ]:
# import gradio as gr

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.faiss import FAISS
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores.base import VectorStoreRetriever

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

from transformers import TextIteratorStreamer
from threading import Thread

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

##test 1

In [ ]:

# Prompt template
template = """Instruction:
You are an AI assistant for answering questions about the provided context.
You are given the following extracted parts of a long document and a question. Provide a detailed answer.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
=======
{context}
=======
Question: {question}
Output:\n"""

QA_PROMPT = PromptTemplate(template=template, input_variables=["question", "context"])

# Load Phi-2 model from hugging face hub
model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.float32, device_map="auto", trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# sentence transformers to be used in vector store
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-base-v4",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False},
)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Returns a faiss vector store retriever given a txt file
def prepare_vector_store_retriever(filename):
    # Load data
    loader = UnstructuredFileLoader(filename)
    raw_documents = loader.load()

    # Split the text
    # text_splitter = CharacterTextSplitter(
    #     separator="\n\n", chunk_size=800, chunk_overlap=0, length_function=len
    # )
    text_splitter = RecursiveCharacterTextSplitter( chunk_size=800, chunk_overlap=0, separators=[" ", ",", "\n"])

    documents = text_splitter.split_documents(raw_documents)

    # Creating a vectorstore
    vectorstore = FAISS.from_documents(
        documents, embeddings, distance_strategy=DistanceStrategy.DOT_PRODUCT
    )

    return VectorStoreRetriever(vectorstore=vectorstore, search_kwargs={"k": 2})


# Retrieveal QA chian
def get_retrieval_qa_chain(text_file, hf_model):
    retriever = default_retriever
    if text_file != default_text_file:
        retriever = prepare_vector_store_retriever(text_file)

    chain = RetrievalQA.from_chain_type(
        llm=hf_model,
        retriever=retriever,
        chain_type_kwargs={"prompt": QA_PROMPT},
    )
    return chain


# Generates response using the question answering chain defined earlier
def generate(question, answer, text_file, max_new_tokens):
    streamer = TextIteratorStreamer(
        tokenizer=tokenizer, skip_prompt=True, skip_special_tokens=True, timeout=300.0
    )
    phi2_pipeline = pipeline(
        "text-generation",
        tokenizer=tokenizer,
        model=model,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        device_map="auto",
        streamer=streamer,
    )

    hf_model = HuggingFacePipeline(pipeline=phi2_pipeline)
    qa_chain = get_retrieval_qa_chain(text_file, hf_model)

    query = f"{question}"

    if len(tokenizer.tokenize(query)) >= 512:
        query = "Repeat 'Your question is too long!'"

    thread = Thread(target=qa_chain.invoke, kwargs={"input": {"query": query}})
    thread.start()

    response = ""
    for token in streamer:
        response += token
        yield response.strip()


# replaces the retreiver in the question answering chain whenever a new file is uploaded
def upload_file(file):
    return file, file

In [ ]:
default_text_file = "/content/23288-i40.txt"
default_retriever = prepare_vector_store_retriever(default_text_file)

In [ ]:
text_file = default_text_file
ques = "What is the role of MDA MnS producer in the management function? [3GPP Release 17]"

# e(question, answer, text_file, max_new_tokens):
generate(ques,"ans",text_file,64)

<generator object generate at 0x7dd6178e5850>

In [ ]:

with gr.Blocks() as demo:
  #   gr.Markdown(
  #       """
  # # Retrieval Augmented Generation with Phi-2: Question Answering demo
  # ### This demo uses the Phi-2 language model and Retrieval Augmented Generation (RAG). It allows you to upload a txt file and ask the model questions related to the content of that file.
  # ### If you don't have one, there is a txt file already loaded, the new Oppenheimer movie's entire wikipedia page. The movie came out very recently in July, 2023, so the Phi-2 model is not aware of it.
  # The context size of the Phi-2 model is 2048 tokens, so even this medium size wikipedia page (11.5k tokens) does not fit in the context window.
  # Retrieval Augmented Generation (RAG) enables us to retrieve just the few small chunks of the document that are relevant to the our query and inject it into our prompt.
  # The model is then able to answer questions by incorporating knowledge from the newly provided document. RAG can be used with thousands of documents, but this demo is limited to just one txt file.
  # """
  #   )

    default_text_file = "/content/23288-i40.txt"
    default_retriever = prepare_vector_store_retriever(default_text_file)

    text_file = gr.State(default_text_file)

    # gr.Markdown(
    #     "## Upload a txt file or Use the Default 'Oppenheimer-movie-wiki.txt' that has already been loaded"
    # )

    # file_name = gr.Textbox(
    #     label="Loaded text file", value=default_text_file, lines=1, interactive=False
    # )
    # upload_button = gr.UploadButton(
    #     label="Click to upload a text file", file_types=["text"], file_count="single"
    # )
    # upload_button.upload(upload_file, upload_button, [file_name, text_file])

    # gr.Markdown("## Enter your question")
    # tokens_slider = gr.Slider(
    #     8,
    #     256,
    #     value=64,
    #     label="Maximum new tokens",
    #     info="A larger `max_new_tokens` parameter value gives you longer text responses but at the cost of a slower response time.",
    # )

    # with gr.Row():
    #     with gr.Column():
    #         print("in column")
    #         ques = gr.Textbox(label="Question", placeholder="Enter text here", lines=3)
    #         print(ques)
    #     with gr.Column():
    #         ans = gr.Textbox(label="Answer", lines=4, interactive=False)
    # with gr.Row():
    #     with gr.Column():
    #         btn = gr.Button("Submit")
    #     with gr.Column():
    #         clear = gr.ClearButton([ques, ans])

    btn.click(fn=generate, inputs=[ques, ans, text_file, 64], outputs=[ans])
    examples = gr.Examples(
        examples=[
            "What is the role of MDA MnS producer in the management function? [3GPP Release 17]",
            "In the plot of the movie, why did Lewis Strauss resent Robert Oppenheimer?",
            "How much money did the Oppenheimer movie make at the US and global box office?",
            "What score did the Oppenheimer movie get on Rotten Tomatoes and Metacritic?",
        ],
        inputs=[ques],
    )

demo.queue().launch()

##test 2

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.vectorstores.faiss import FAISS
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load your model and tokenizer
model_name ="microsoft/phi-2"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

# Set up the text generation pipeline
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Set up document loader, vector store, and retriever
document_loader = UnstructuredFileLoader(file_path="/content/23288-i40.txt")
documents = document_loader.load()

# sentence transformers to be used in vector store
# embedding = HuggingFaceEmbeddings()
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-base-v4",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False},
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:

vector_store = FAISS.from_documents(documents, embedding, distance_strategy=DistanceStrategy.COSINE)
retriever = VectorStoreRetriever(vector_store=vector_store)

# Set up retrieval-based QA chain
qa_chain = RetrievalQA(retriever=retriever, model=text_gen_pipeline)

# Define your question
question = "What is the role of MDA MnS producer in the management function? [3GPP Release 17]"

# Retrieve context and generate answer
context = retriever.retrieve(question)
answer = text_gen_pipeline(question, context=context, max_new_tokens=50)[0]['generated_text']

print("Context:", context)
print("Answer:", answer)


NameError: name 'FAISS' is not defined